# Part 8 - Introdução a Planos


### Contexto 

Aqui nós introduzimos um objeto que é crucial para escalar a indústria do Aprendizado Federado: Planos. Isso reduz dramaticamente o uso de largura de banda, permite esquecas assíncronos e da mais autonomia para dispositivos remotos. O conceiro original do plano pode ser encontrado no artigo [Towards Federated Learning at Scale: System Design](https://arxiv.org/pdf/1902.01046.pdf), porém com algumas adaptações para as nossas necessidades no PySyft.

Um plano tem a intenção de salvar uma sequência de operações do torch, como uma função, mas permitindo enviar essa sequência de operações para um worker remoto e manter a referência para isso. Dessa forma, para computar remotamente essa sequência de $n$ operações em alguma entrada referenciada através desses ponteiros, ao invés de mandar $n$ mensagens, você precisa agora enviar apenas uma única mensagem com as referências para o plano e para os ponteiros. Você pode também fornecer tensores com sua funcão (que chamamos _state tensors_) para ter as funcionalidades extendidas. Planos podem ser vistos tanto como uma função que você pode enviar, como uma classe que também pode ser enviada e executada remotamente. Portanto, para usuários de nível alto, a noção de plano desaparece e é substituída por um recurso mágico que permite enviar para workers remotos funcões arbitrárias que contém uma sequência de funções torch.

Uma coisa a se notar é que a classe de funções que você quer transformar em planos é atualmente limitada para sequências de operações _hook_ (gancho) do torch exclusivamente. Em particular, isso exclúi estruturas lógicas como `if`, `for` e `while`, mesmo que estejamos trabalhando para ter soluções em breve. _Para ser completamente preciso, você pode usá-los mas o caminho lógico que você toma (primeiro `if` é falso e 5 laços `for`, por exemplo) na primeira computação do seu plano será levado por todas as próximas computações, cujo queremos evitar nos casos majoritárias._

Autores:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - GitHub: [@LaRiffle](https://github.com/LaRiffle)
- Bobby Wagner - Twitter [@bobbyawagner](https://twitter.com/bobbyawagner) - GitHub: [@robert-wagner](https://github.com/robert-wagner)
- Marianne Monteiro - Twitter [@hereismari](https://twitter.com/hereismari) - GitHub: [@mari-linhares](https://github.com/mari-linhares)	

Tradutor:
- João Lucas - GitHub: [@joaolcaas](https://github.com/joaolcaas) 

### Importações e especificações do modelo

Primeiro faremos as importações oficiais.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

E então aqueles específicos do PySyft, com uma importante observação: **o worker local não deve ser um worker cliente**. Workers não clientes podem salvar objetos e precisamos dessa habilidade para usar um plano.

In [3]:
import syft as sy  # importe a biblioteca PySyft
hook = sy.TorchHook(torch)  # hook PyTorch ie adiciona funcionalidades extras 

# IMPORTANTE: worker local não deve ser um worker cliente
hook.local_worker.is_client_worker = False


server = hook.local_worker

Para ser consistente com o conceito fornecido no artigo referenciado, definimos _workers_ remotos ou _devices_, provendo eles com algum dado. 

In [4]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

### Exemplo Básico

Vamos definir uma função que queremos transformar em um plano. Para isso, apenas adiciona-se um _decorator_ acima da definição da função.

In [5]:
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

Vamos verificar.
Sim, agora nós temos um plano!

In [6]:
plan_double_abs

<Plan plan_double_abs id:89796211423 owner:me>

Para usar um plano, precisamos de duas coisas: contruir um plano _(i.e. registrar uma sequência de operações presentes na função)_ e mandar isso para um worker / device. Felizmente você pode fazer isso de uma maneira muito fácil.

#### Construindo um plano

Para construir um plano, você só precisa chamar isso em algum dado.

Vamos primeiro pegar a referência para algum dado remoto: uma requisição é enviada para a rede e o ponteiro de referência é retornado.

In [7]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

(Wrapper)>[PointerTensor | me:33127708416 -> device_1:47766305516]
	Tags: input_data 
	Shape: torch.Size([2])

Se dissermos para um plano que ele deve ser executado remotamente no dispositivo `location:device_1`, isso retornará um erro pois o plano não foi construído ainda.

In [8]:
plan_double_abs.is_built

False

In [9]:
# Falha acontece se mandar um plano não construído
try:
    plan_double_abs.send(device_1)
except RuntimeError as error:
    print(error)

A plan needs to be built before being sent to a worker.


Para construir um plano só precisamos chamar `build` no plano e passar os argumentos necessários para a execução de um plano (a.k.a algum dado). Quando um plano é construído, todos os comandos são executados sequencialmente por um worker local, capturados pelo plano e então salvos no atributo `readable_plan`!

In [10]:
plan_double_abs.build(torch.tensor([1., -2.]))

In [11]:
plan_double_abs.is_built

True

Agora, se tentarmos enviar um plano, irá funcionar!

In [12]:
# Essa célula é executada com sucesso
pointer_plan = plan_double_abs.send(device_1)
pointer_plan

[PointerPlan | me:85123354948 -> device_1:89796211423]

Como nos tensores, podemos ver um ponteiro para o objeto que foi enviado, sendo chamado simplesmente de `PointerPlan`.

Um lembrete importante é que, quando um plano é construído, nós de antemão pré-definimos os id(s) dos resultados das computações que devem ser salvos. Isso irá permitir enviar os comandos de forma assíncrona, já ter uma referência para de um resultado virtual e continuar as computações locais sem ter que esperar os resultados remotos serem computados. Uma aplicação importante é quando você exige a computação de um grupo no device_1 e não quer esperar pelo fim dessa computação para mandar outro grupo de computação para o device_2

#### Executando um Plano Remotamente

Nós agora podemos executar um plano remotamente chamando um ponteiro para um plano com um ponteiro para algum dado. Isso emite um comando para executar esse plano remotamente, sendo assim, a localização pré-definida da saída do plano agora contém o resultado (lembrese que  pré-definimos a localização do resultado antes da computação). Isso também requer uma única rodada de comunicação.

O resultado é simplesmente um ponteiro, assim como quando você chamada uma função do Torch enganchada (hooked).

In [ ]:
pointer_to_result = pointer_plan(pointer_to_data)
print(pointer_to_result)

E você pode simplesmentes pegar o valor de volta.

In [ ]:
pointer_to_result.get()

### Towards a concrete example

Mas nós queremos que Planos sejam aplicados para aprendizago profundi e federado, certo? Então vamos dar uma olhada em um exemplo ligeiramente mais complicado, usando rede neural que certamente você deve estar ansioso a usá-lo.
Note que estamos agora transformando uma classe em um Plano. Para isso, nós herdamos a nossa classe de sy.Plan (ao invés de herdar de nn.Module)

In [13]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)


In [14]:
net = Net()

In [15]:
net

<Net Net id:58498301328 owner:me>

Vamos construir um plano usando dados de exemplo.

In [16]:
net.build(torch.tensor([1., 2.]))

Agora enviamos o Plano para um worker remoto.

In [18]:
pointer_to_net = net.send(device_1)
pointer_to_net

[PointerPlan | me:66386943511 -> device_1:58498301328]

Vamos recuperar alguns dados remotos.

In [19]:
pointer_to_data = device_1.search('input_data')[0]

A sintaxe é como a execução sequencial remota normal, isso é, como uma execução local. Mas comparada a uma execução remota clássica, tem apenas uma rodada de comunicação para cada execução.

In [20]:
pointer_to_result = pointer_to_net(pointer_to_data)
pointer_to_result

(Wrapper)>[PointerTensor | me:87899785368 -> device_1:93434720381]

E recebemos o resultado como sempre!

In [ ]:
pointer_to_result.get()

Et voilà! Nós vimos como dramaticamente reduzir a comunicação entre worker local (ou servidor) e os dispositivos remotos.

### Troca entre Workers

Uma característica importante que queremos ter é usar o mesmo plano para vários workers, que podemos mudar dependendo da parte remota do dado que estamos considerando.
Em especial, não queremos reconstruir o plano cada vez que temos que mudar de worker. Vamos ver como fazer isso usando o exemplo visto ateriormente com a nossa pequena rede.

In [ ]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [ ]:
net = Net()

# Build plan
net.build(torch.tensor([1., 2.]))

Aqui são executados os passos principais

In [ ]:
pointer_to_net_1 = net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_to_net_1(pointer_to_data)
pointer_to_result.get()

E você pode construir outro PointerPlan a partir do mesmo plano, sendo a sintaxe a mesma para executar o plano remotamente em outro dispositivo.

In [ ]:
pointer_to_net_2 = net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = pointer_to_net_2(pointer_to_data)
pointer_to_result.get()

> Note: Atualmente, com a classe Plano, você pode apenas usar um único método e tem que nomear como "forward"

### Construindo planos automaticamente que são funcções

Para funcões (`@` `sy.func2plan`) nós podemos automaticamente construir um plano sem e necessidada de explicitar a chamada do `build`, uma vez que no momento da criação o plano já é criado.

Para usar essa funcionalidade a única coisa que você deve mudar na criação do plano é definir um argumento para o decorator chamado `args_shape` que deve ser uma lista que contém os shapes de cada argumento.

In [ ]:
@sy.func2plan(args_shape=[(-1, 1)])
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

plan_double_abs.is_built

O parâmetro `args_shape` é usado internamente para criar tensores de exemplo com o formato em qual foi passado para construir o plano.

In [ ]:
@sy.func2plan(args_shape=[(1, 2), (-1, 2)])
def plan_sum_abs(x, y):
    s = x + y
    return torch.abs(s)

plan_sum_abs.is_built

Também é possível prover
You can also provide state elements to functions!

In [ ]:
@sy.func2plan(args_shape=[(1,)], state=(torch.tensor([1]), ))
def plan_abs(x, state):
    bias, = state.read()
    x = x.abs()
    return x + bias

In [ ]:
pointer_plan = plan_abs.send(device_1)
x_ptr = torch.tensor([-1, 0]).send(device_1)
p = pointer_plan(x_ptr)
p.get()

Para aprender mais sobre isso, você pode descobrir como usar Planos com Protocolos no tutorial Part 8 bis!

### Dê-nos uma estrela em nosso repo do PySyft no GitHub

A maneira mais fácil de ajudar nossa comunidade é adicionando uma estrela nos nossos repositórios! Isso ajuda a aumentar a conscientização sobre essas ferramentas legais que estamos construindo.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Veja nossos tutoriais no GitHub!

Fizemos tutoriais muito bons para entender melhor como deve ser a Aprendizagem Federada e a proteção de Privacidade, e como estamos construindo as coisas básicas que precisamos para fazer com que isso aconteça.

- [Tutoriais do PySyft](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)

### Junte-se ao Slack!

A melhor maneira de manter-se atualizado sobre os últimos avanços é se juntar à nossa comunidade! 

- [http://slack.openmined.org](http://slack.openmined.org)

### Contribua com o projeto!

A melhor maneira de contribuir para a nossa comunidade é se tornando um contribuidor do código! A qualquer momento, você pode acessar a página de *Issues* (problemas) do PySyft no GitHub e filtrar por "Projetos". Isso mostrará todas as etiquetas (tags) na parte superior, com uma visão geral de quais projetos você pode participar! Se você não deseja ingressar em um projeto, mas gostaria de codificar um pouco, também pode procurar mais mini-projetos "independentes" pesquisando problemas no GitHub marcados como "good first issue".

- [Etiquetados como Good First Issue](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Doar

Se você não tem tempo para contribuir com nossa base de códigos, mas ainda deseja nos apoiar, também pode se tornar um Apoiador em nosso Open Collective. Todas as doações vão para hospedagem na web e outras despesas da comunidade, como hackathons e meetups!

[Página do Open Collective do OpenMined](https://opencollective.com/openmined)